In [1]:
import tensorflow as tf

# Step 1: Load a pre-trained MobileNetV2 model from tf.keras.applications
model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))

# Step 2: Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Disable experimental features to avoid MLIR errors
converter.experimental_new_converter = False  # Use the old converter for stability
tflite_model = converter.convert()

# Step 3: Save the converted .tflite model
tflite_model_path = "mobilenetv2_model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"Model successfully converted to {tflite_model_path}")

INFO:tensorflow:Assets written to: /var/folders/hb/fpmv_swd6pb2nt84zqhq62fh0000gq/T/tmppjtknaw_/assets


INFO:tensorflow:Assets written to: /var/folders/hb/fpmv_swd6pb2nt84zqhq62fh0000gq/T/tmppjtknaw_/assets
2024-10-02 12:50:15.417709: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-02 12:50:15.417824: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-10-02 12:50:15.767509: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-02 12:50:15.767591: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


Model successfully converted to mobilenetv2_model.tflite


In [2]:
import tensorflow as tf

# Load and inspect the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenetv2_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input details: {input_details}")
print(f"Output details: {output_details}")

Input details: [{'name': 'input_1', 'index': 1, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'Identity', 'index': 0, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenetv2_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load a sample image, resize to model input shape, and preprocess
img = Image.open("sample_image.jpg").resize((224, 224))
input_data = np.expand_dims(np.array(img) / 255.0, axis=0).astype(np.float32)

# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output from the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(f"Output Data: {output_data}")

predicted_class = np.argmax(output_data)
print(f"Predicted Class Index: {predicted_class}")

Output Data: [[5.32124133e-04 4.11124813e-04 1.55205710e-03 9.78591619e-04
  7.98049557e-04 4.80113726e-04 3.77060380e-03 1.29565757e-04
  3.73081217e-04 2.70167511e-04 4.24647878e-04 5.69683325e-05
  8.29708442e-05 3.74735449e-04 1.16127703e-04 9.86763116e-05
  2.04535885e-04 1.33691428e-04 1.89344515e-04 7.75614317e-05
  6.36670855e-04 1.75372767e-03 3.36875004e-04 7.63901975e-04
  3.63954430e-04 8.05532734e-04 1.07493310e-03 7.84753996e-04
  6.43054082e-04 2.64178670e-04 3.36778932e-04 3.47278983e-04
  4.99603921e-04 3.81212994e-05 9.84676648e-04 4.20603901e-05
  9.80644545e-05 4.34587710e-05 1.91896717e-04 1.98271140e-04
  2.35961037e-04 6.11893949e-04 2.10698548e-04 1.45547136e-04
  2.37128901e-04 2.51650956e-04 3.08292016e-04 2.12225554e-04
  1.56842405e-04 8.17754990e-05 8.57819978e-05 3.71271599e-05
  3.95701383e-04 4.54869762e-04 3.19997373e-04 1.11554109e-04
  1.14408460e-04 5.19055640e-04 9.19196114e-04 1.94342414e-04
  4.95813329e-05 4.26325947e-04 5.47912146e-04 5.92856726

In [7]:
import tensorflow as tf

# Load a pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))

# Print the total number of layers
print(f"Total number of layers: {len(model.layers)}")

# List all the layer names for detailed inspection
for i, layer in enumerate(model.layers):
    print(f"Layer {i+1}: {layer.name} ({layer.__class__.__name__})")

Total number of layers: 156
Layer 1: input_2 (InputLayer)
Layer 2: Conv1 (Conv2D)
Layer 3: bn_Conv1 (BatchNormalization)
Layer 4: Conv1_relu (ReLU)
Layer 5: expanded_conv_depthwise (DepthwiseConv2D)
Layer 6: expanded_conv_depthwise_BN (BatchNormalization)
Layer 7: expanded_conv_depthwise_relu (ReLU)
Layer 8: expanded_conv_project (Conv2D)
Layer 9: expanded_conv_project_BN (BatchNormalization)
Layer 10: block_1_expand (Conv2D)
Layer 11: block_1_expand_BN (BatchNormalization)
Layer 12: block_1_expand_relu (ReLU)
Layer 13: block_1_pad (ZeroPadding2D)
Layer 14: block_1_depthwise (DepthwiseConv2D)
Layer 15: block_1_depthwise_BN (BatchNormalization)
Layer 16: block_1_depthwise_relu (ReLU)
Layer 17: block_1_project (Conv2D)
Layer 18: block_1_project_BN (BatchNormalization)
Layer 19: block_2_expand (Conv2D)
Layer 20: block_2_expand_BN (BatchNormalization)
Layer 21: block_2_expand_relu (ReLU)
Layer 22: block_2_depthwise (DepthwiseConv2D)
Layer 23: block_2_depthwise_BN (BatchNormalization)
Layer

In [13]:
import tensorflow as tf

# Load a pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))

# Helper function to calculate size in bytes of a tensor
def get_tensor_size(tensor):
    if tensor is None:
        return 0
    # Number of elements * size of each element (assuming float32)
    return tf.size(tensor).numpy() * tensor.dtype.size

# Function to calculate the size of the activations using the output shape
def get_activation_size(output_shape, dtype=tf.float32):
    # Check if output_shape is valid and not a tuple with None values
    if output_shape is None or not isinstance(output_shape, tuple) or None in output_shape:
        return 0
    try:
        # Multiply all dimensions to get the number of elements (ignoring batch size)
        num_elements = 1
        for dim in output_shape:
            if dim is not None:  # Ignore undefined dimensions
                num_elements *= dim
        return num_elements * dtype.size  # Size of each element (4 bytes for float32)
    except Exception as e:
        print(f"Error calculating activation size for shape {output_shape}: {e}")
        return 0

# Display total number of layers in the model
print(f"Total number of layers: {len(model.layers)}")

# Loop through each layer to calculate weight and activation sizes
for i, layer in enumerate(model.layers):
    weight_size = 0
    activation_size = 0

    # Calculate the size of weights (weights + biases)
    for weight in layer.weights:
        weight_size += get_tensor_size(weight)

    # Calculate the size of the activations using the output shape
    activation_size = get_activation_size(layer.output_shape)

    # Ensure `activation_size` is an integer
    if not isinstance(activation_size, int):
        print(f"Warning: Activation size for layer {layer.name} is not an integer: {activation_size}")
        activation_size = 0

    # Calculate the total size for the layer
    total_layer_size = weight_size + activation_size

    # Print the layer details along with memory sizes
    print(f"Layer {i+1}: {layer.name} ({layer.__class__.__name__})")
    print(f"  - Weights: {weight_size / 1024:.2f} KB, Activations: {activation_size / 1024:.2f} KB, Total: {total_layer_size / 1024:.2f} KB\n")

Total number of layers: 156
Layer 1: input_8 (InputLayer)
  - Weights: 0.00 KB, Activations: 0.00 KB, Total: 0.00 KB

Layer 2: Conv1 (Conv2D)
  - Weights: 3.38 KB, Activations: 0.00 KB, Total: 3.38 KB

Layer 3: bn_Conv1 (BatchNormalization)
  - Weights: 0.50 KB, Activations: 0.00 KB, Total: 0.50 KB

Layer 4: Conv1_relu (ReLU)
  - Weights: 0.00 KB, Activations: 0.00 KB, Total: 0.00 KB

Layer 5: expanded_conv_depthwise (DepthwiseConv2D)
  - Weights: 1.12 KB, Activations: 0.00 KB, Total: 1.12 KB

Layer 6: expanded_conv_depthwise_BN (BatchNormalization)
  - Weights: 0.50 KB, Activations: 0.00 KB, Total: 0.50 KB

Layer 7: expanded_conv_depthwise_relu (ReLU)
  - Weights: 0.00 KB, Activations: 0.00 KB, Total: 0.00 KB

Layer 8: expanded_conv_project (Conv2D)
  - Weights: 2.00 KB, Activations: 0.00 KB, Total: 2.00 KB

Layer 9: expanded_conv_project_BN (BatchNormalization)
  - Weights: 0.25 KB, Activations: 0.00 KB, Total: 0.25 KB

Layer 10: block_1_expand (Conv2D)
  - Weights: 6.00 KB, Activati

In [14]:
import tensorflow as tf

# Load a pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))

# Helper function to calculate size in bytes of a tensor
def get_tensor_size(tensor):
    if tensor is None:
        return 0
    # Number of elements * size of each element (assuming float32)
    return tf.size(tensor).numpy() * tensor.dtype.size

# Function to calculate the size of the activations using the output shape
def get_activation_size(output_shape, dtype=tf.float32):
    if output_shape is None or not isinstance(output_shape, tuple) or None in output_shape:
        return 0
    try:
        # Multiply all dimensions to get the number of elements (ignoring batch size)
        num_elements = 1
        for dim in output_shape:
            if dim is not None:  # Ignore undefined dimensions
                num_elements *= dim
        return num_elements * dtype.size  # Size of each element (4 bytes for float32)
    except Exception as e:
        print(f"Error calculating activation size for shape {output_shape}: {e}")
        return 0

# Display total number of layers in the model
print(f"Total number of layers: {len(model.layers)}")

# Define functional blocks for coarse-grained breakdown
functional_blocks = {
    'Initial Conv Block': list(range(0, 5)),      # Conv2D, BatchNorm, ReLU6
    'Residual Block 1': list(range(5, 15)),       # Block 1: expand, depthwise, project
    'Residual Block 2': list(range(15, 27)),      # Block 2: expand, depthwise, project
    'Residual Block 3': list(range(27, 43)),      # Block 3: expand, depthwise, project
    'Residual Block 4': list(range(43, 61)),      # Block 4: expand, depthwise, project
    'Bottleneck Block': list(range(61, 69)),      # Bottleneck layers
    'Final Conv Block': list(range(69, 152)),     # Final convolution + BatchNorm + Activation
    'Output Layer': list(range(152, 156))         # GlobalAveragePooling2D + Dense
}

# Calculate and display memory usage for each functional block
for block_name, layer_indices in functional_blocks.items():
    block_weight_size = 0
    block_activation_size = 0

    # Aggregate sizes for all layers in the functional block
    for index in layer_indices:
        layer = model.layers[index]
        # Calculate weight size for the current layer
        for weight in layer.weights:
            block_weight_size += get_tensor_size(weight)
        
        # Calculate activation size
        activation_size = get_activation_size(layer.output_shape)
        if isinstance(activation_size, int):
            block_activation_size += activation_size

    # Calculate total size for the functional block
    block_total_size = block_weight_size + block_activation_size
    print(f"{block_name}:")
    print(f"  - Weights: {block_weight_size / 1024:.2f} KB, Activations: {block_activation_size / 1024:.2f} KB, Total: {block_total_size / 1024:.2f} KB\n")

Total number of layers: 156
Initial Conv Block:
  - Weights: 5.00 KB, Activations: 0.00 KB, Total: 5.00 KB

Residual Block 1:
  - Weights: 15.12 KB, Activations: 0.00 KB, Total: 15.12 KB

Residual Block 2:
  - Weights: 46.31 KB, Activations: 0.00 KB, Total: 46.31 KB

Residual Block 3:
  - Weights: 102.31 KB, Activations: 0.00 KB, Total: 102.31 KB

Residual Block 4:
  - Weights: 98.50 KB, Activations: 0.00 KB, Total: 98.50 KB

Bottleneck Block:
  - Weights: 170.50 KB, Activations: 0.00 KB, Total: 170.50 KB

Final Conv Block:
  - Weights: 8362.50 KB, Activations: 0.00 KB, Total: 8362.50 KB

Output Layer:
  - Weights: 5023.91 KB, Activations: 0.00 KB, Total: 5023.91 KB

